In [212]:
import json
import pandas as pd
import numpy as np
import sys
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
from collections import Counter
import Levenshtein
import re
import regex

tweets = pd.read_json("Data\gg2013.json")  
print("Done")

Done


In [185]:
#### Global variables
awards = ["best screenplay - motion picture", "best director - motion picture",
          "best performance by an actress in a television series - comedy or musical",
          "best foreign language film", "best performance by an actor in a supporting role in a motion picture",
          "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
          "best motion picture - comedy or musical",
          "best performance by an actress in a motion picture - comedy or musical",
          "best mini-series or motion picture made for television",
          "best original score - motion picture", "best performance by an actress in a television series - drama",
          "best performance by an actress in a motion picture - drama", "cecil b. demille award",
          "best performance by an actor in a motion picture - comedy or musical", "best motion picture - drama",
          "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television",
          "best performance by an actress in a supporting role in a motion picture", "best television series - drama",
          "best performance by an actor in a mini-series or motion picture made for television",
          "best performance by an actress in a mini-series or motion picture made for television",
          "best animated feature film", "best original song - motion picture",
          "best performance by an actor in a motion picture - drama", "best television series - comedy or musical",
          "best performance by an actor in a television series - drama",
          "best performance by an actor in a television series - comedy or musical"] 

key_word_award_mapping = {
        "best screenplay - motion picture": ["screenplay", "motion", "^(?!.*actor).*$", "^(?!.*actress).*$"], 
        "best director - motion picture": ["director", "motion", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actress in a television series - comedy or musical" : ["actress", "comedy|musical", "television|tv|tele|series", "^(?!.*support).*$"],
        "best foreign language film" : ["foreign"],
        "best performance by an actor in a supporting role in a motion picture" : ["actor", "support", "motion"],
        "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television" : ["actress", "support", "television|tv|tele|series|film", "mini|film"],
        "best motion picture - comedy or musical" : ["motion", "comedy|musical", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actress in a motion picture - comedy or musical" : ["actress", "motion", "comedy|musical", "^(?!.*support).*$"],
        "best mini-series or motion picture made for television" : ["television|tv|series", "mini|film", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best original score - motion picture" : ["score", "motion", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actress in a television series - drama" : ["actress", "television|tv|tele|series", "drama", "^(?!.*support).*$"],
        "best performance by an actress in a motion picture - drama" : ["actress", "motion", "drama", "^(?!.*support).*$"],
        "cecil b. demille award" : ["cecil", "demille"],
        "best performance by an actor in a motion picture - comedy or musical" : ["actor", "motion", "comedy|musical", "^(?!.*support).*$"], 
        "best motion picture - drama" : ["motion", "drama", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television" : ["support", "actor", "television|tv|tele|series|film", "mini|film"],
        "best performance by an actress in a supporting role in a motion picture" : ["actress", "support", "motion"],
        "best television series - drama" : ["television|tv|series", "drama", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actor in a mini-series or motion picture made for television": ["actor", "television|tv|series", "mini|film" "^(?!.*support).*$"],
        "best performance by an actress in a mini-series or motion picture made for television" : ["actress", "television|tv|tele|series|film", "mini|film", "^(?!.*support).*$"],
        "best animated feature film" : ["animated", "film"],
        "best original song - motion picture" : ["original", "song", "motion", "^(?!.*actor).*$", "^(?!.*actress).*$", "^(?!.*by).*$"],
        "best performance by an actor in a motion picture - drama" : ["actor", "motion", "drama", "^(?!.*support).*$"],
        "best television series - comedy or musical" : ["television|tv|series", "comedy|musical", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actor in a television series - drama" : ["actor", "television|tv|series", "drama", "^(?!.*support).*$"],
        "best performance by an actor in a television series - comedy or musical" : ["actor", "television|tv|series", "comedy|musical", "^(?!.*support).*$"]    } 
    
tokens_to_ignore = ["RT", "@", "golden globe", "goldenglobe", "globe", "globes", "golden", "hosts", "http", "#"]

In [84]:
###Sequence to Get Potential Answers
def getSolutionsFromPositions(doc, pos):
    consecutives = []
    current = []
    for elt in pos:
        if len(current) == 0:
            current.append(elt)
        else:
            if current[-1] == elt - 1:
                current.append(elt)
            else:
                consecutives.append(current)
                current = [elt]
        if (len(current) != 0):
            consecutives.append(current)
    if ([doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives] is None):
        return []
    else:
        return [doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives]

In [210]:
def host():
    print("starting")
    hosts = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        if ("host" in tweet.lower()):
            doc = nlp(tweet)
            #Grabs proper pronouns (Removes certain key phrases too)
            pos = [tok.i for tok in doc if (tok.pos_ == "PROPN" and not any(i in tok.text.lower() for i in tokens_to_ignore))]
            properPronouns = getSolutionsFromPositions(doc, pos)
            
            # Only want Phrases with 2 words (to represent a full name)
            for h in (properPronouns):
                lowerCase = h.text.lower()
                if (len(h.text.split()) != 2):
                    continue
                if lowerCase in hosts:
                    hosts[lowerCase] += 1
                else:
                    hosts[lowerCase] = 1
                        
    #Find the top 1-2 Names, those will be the people mentioned the most with the word host
    host_list = sorted(hosts.keys(), key=hosts.get, reverse=True)
    print(host_list)
    p = np.percentile(list(hosts.values()),99)
    answer = []
    for key in host_list:
        if (hosts[key] > p):
            answer.append(key)
    if (len(answer) >= 2):
        return answer[:2]
    else:
        return answer[0]
    

In [211]:
print(host())

starting
['tina fey', 'amy poehler', 'kristen wiig', 'opening monologue', 'sexual nightmare', 'amy poelher', 'amy pohler', 'quentin tarantino', 'quentin tarantin', 'will ferrell', 'ricky gervais', 'jodie foster', 'kristin wiig', 'maggie smith', 'james cameron', 'anne hathaway', 'loving tina', 'paul rudd', 'will farrell', 'maggie smi', 'jay leno', 'bill clinton', 'open post', 'kristen wigg', 'dog president', 'jennifer lawrence', 'james franco', 'kristen wig', 'amy poeler', 'bill murray', 'love tina', 'seth macfarlane', 'kevin costner', 'ed harris', 'tina fay', 'japanese cojo', 'red carpet', 'glenn close', 'taylor swift', 'george clooney', 'jimmy fallon', 'drinking game', 'tiny fey', 'tony mendez', 'ryan seacrest', 'kristin wigg', 'quentin tar', 'quentin t', 'mel gibson', 'thinking tina', 'today show', 'tine fey', 'omg tina', 'amy pholer', 'tina faye', 'knew amy', 'queen amy', 'hunger games', 'amy polar', 'amy p.', 'daniel day', 'bravo tina', 'bruce hosted', 'w magazine', 'amy phoeler', 

In [208]:
### Not Working Well
def award_names():
    print("STARTING")
    awardNames = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        if ("best " in tweet.lower()):# and(("nominated" in tweet.lower()) or ("won" in tweet.lower()))):                       
            doc = nlp(tweet)
            nouns = ["PROPN", "NOUN"]
            pos = [tok.i for tok in doc if ((tok.pos_ in nouns) or (tok.pos_ == "ADJ" and tok.text.lower() == "best")) and not any(i in tok.text.lower() for i in tokens_to_ignore)]
            words = getSolutionsFromPositions(doc, pos)
            for h in (words):
                lowerCase = h.text.lower()
                if (lowerCase[0:5] != "best "):
                    continue
                if lowerCase in awardNames:
                    awardNames[lowerCase] += 1
                else:
                    awardNames[lowerCase] = 1
                        
                                            
    #Find the top 1-2 Names, those will be the people mentioned the most with the word best
    award_list = sorted(awardNames.keys(), key=awardNames.get, reverse = True)
    answer = []
    for i in award_list:
        answer.append(i)
    return answer[:27]

In [209]:
print(award_names())

STARTING
['best actor', 'best actress', 'best director', 'best motion picture', 'best picture', 'best supporting actress', 'best supporting actor', 'best original song', 'best screenplay', 'best tv series', 'best song', 'best part', 'best motion picture drama', 'best performance', 'best drama', 'best movie', 'best comedy', 'best speech', 'best exotic marigold hotel', 'best television series', 'best foreign film', 'best show', 'best thing', 'best actress drama winner', 'best moments', 'best acceptance speech', 'best foreign language film']


In [ ]:
def fuzzy_match(base, candidate, threshold):
    dist = Levenshtein.distance(base, candidate)
    base_len = len(base_str)
    return (dist <= round(base_len * threshold))

In [214]:
def award_winners():
    print("STARTING")
    phrases_to_ignore = ["motion picture", "best", "hbo"]
    winners = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
            
        if (("won" in tweet.lower() or "congrat" in tweet.lower() or "winner" in tweet.lower() or "goes to" in tweet.lower()) and ("best" in tweet.lower() or "cecil" in tweet.lower())):
            doc = nlp(tweet)
            for award in awards:
                go = True
                for i in key_word_award_mapping[award]:
                    if not bool(re.search(i, tweet.lower())):
                        go = False
                        break
                if not go:
                    continue
                
                movies_tv_pos = ["PROPN", "NOUN", "ADJ", "AUX", "VERB"]
                if ("act" in award or "cecil" in award):
                    pos = [tok.i for tok in doc if (tok.pos_ == "PROPN" and not any(i in tok.text.lower() for i in tokens_to_ignore))] #maybe include not congrat
                else:
                    pos = [tok.i for tok in doc if ((tok.pos_ in movies_tv_pos or tok.text.lower() == "the" or tok.text.lower() == "of") and not any(i in tok.text.lower() for i in tokens_to_ignore))]
            
    
            
            
                properPronouns = getSolutionsFromPositions(doc, pos)
                for h in (properPronouns):
                    if ("hbo" in h.text.lower() or "best" in h.text.lower() or "motion picture" in h.text.lower() or "winners" == h.text.lower() or "tv" == h.text.lower()):
                        continue
                    if (h.text.lower() in award):
                        continue
                    if (("director" in award or "cecil" in award or "act" in award) and (len(h.text.split()) != 2 and len(h.text.split()) != 3)):
                        continue
                    if award in winners:
                        if h.text.lower() in winners[award]:
                            winners[award][h.text.lower()] += 1
                        else:
                            winners[award][h.text.lower()] = 1
                    else:
                        winners[award] = {}
                        winners[award][h.text.lower()] = 1

    winn = {}
    for x in winners:
        winn[x] = max(winners[x], key=winners[x].get)
    print("done")

    print(winners['best original score - motion picture'])
    
    print(winn)
    for i in awards:
        if i not in winn.keys():
            print(i)
award_winners()

STARTING
done
{"i'm": 2, 'happy': 1, 'winner': 4, 'mychael danna': 5, 'life of pi': 3}
{'best television series - drama': 'homeland', 'best performance by an actor in a supporting role in a motion picture': 'christoph waltz', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': 'maggie smith', 'best performance by an actress in a supporting role in a motion picture': 'anne hathaway', 'best mini-series or motion picture made for television': 'game change', 'best performance by an actress in a mini-series or motion picture made for television': 'julianne moore', 'best performance by an actor in a mini-series or motion picture made for television': 'kevin costner', 'best performance by an actor in a television series - drama': 'damian lewis', 'best performance by an actress in a television series - drama': 'claire danes', 'best original score - motion picture': 'mychael danna', 'best original song - motion picture': 'skyfall